In [1]:
from pathlib import Path
from pyraphtory.context import PyRaphtory
from pyraphtory.vertex import Vertex
from pyraphtory.steps import Iterate, Step
!curl -o /tmp/lotr.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52206  100 52206    0     0   366k      0 --:--:-- --:--:-- --:--:--  386k


In [2]:
pr = PyRaphtory(spout_input=Path('/tmp/lotr.csv'), builder_script=Path('builder.py'),  builder_class='LotrGraphBuilder', mode='batch', logging=False).open()
rg = pr.graph()

b'WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.\n'
b'11:40:22.016 [io-compute-5] DEBUG com.raphtory.internals.management.python.PythonGraphBuilder - Started PythonGraphBuilder\n'
b'11:40:23.082 [io-compute-5] DEBUG com.raphtory.utils.FileUtils$ - Found single file /tmp/lotr.csv matching criteria.\n'
b"11:40:23.083 [io-compute-5] TRACE com.raphtory.utils.FileUtils$ - Creating temp folder '/tmp/raphtory-file-spout/4301136767636149901'.\n"
b"11:40:23.083 [io-compute-5] DEBUG com.raphtory.utils.FileUtils$ - Temporary directory '/tmp/raphtory-file-spout/4301136767636149901' successfully created.\n"
b"11:40:23.084 [io-compute-5] DEBUG com.raphtory.spouts.FileSpout - Spout: Found a new file '/lotr.csv' to process.\n"
b"11:40:23.084 [io-compute-5] TRACE com.raphtory.utils.FileUtils$ - Creating temp folder '/tmp/raphtory-file-spout/4301136767636149901'.\n"
b"11:40:23.084 [io-compute-5] TRACE com.raphtory.utils.FileUtils$ - Temporary directory '/t

In [3]:
local_sink = pr.local_sink()

cols = ["inDegree", "outDegree", "degree","triangleCount","prlabel","cclabel","twoHopPaths"]

# g = rg.at(32674)
# print(type(g))

tracker = rg.at(32674) \
                .past() \
                .transform(pr.page_rank())\
                .transform(pr.connected_components())\
                .transform(pr.degree())\
                .transform(pr.two_hops_path(set([])))\
                .transform(pr.local_triangle_count())\
                .select(cols) \
                .write_to(local_sink)\
                .wait_for_job()


pr.results_to_dataframe(local_sink.results(),cols)



,timestamp,window,name,inDegree,outDegree,degree,triangleCount,prlabel,cclabel,twoHopPaths
0,32674,None,Hirgon,0,2,2,1,0.150000,-8637342647242242534,"[[Gandalf, Galadriel], [Gandalf, Thengel], [Ga..."
1,32674,None,Hador,2,1,3,3,0.248073,-8637342647242242534,"[[Húrin, Elessar], [Húrin, Faramir]]"
2,32674,None,Horn,3,1,4,6,0.281897,-8637342647242242534,NaN
3,32674,None,Galadriel,16,6,19,93,1.202763,-8637342647242242534,"[[Sam, Mablung], [Sam, Peregrin], [Aragorn, Sa..."
4,32674,None,Isildur,0,18,18,55,0.150000,-8637342647242242534,"[[Frodo, Mablung], [Théoden, Gram], [Gollum, D..."
...,...,...,...,...,...,...,...,...,...,...
134,32674,None,Faramir,29,3,29,132,4.614496,-8637342647242242534,"[[Elrond, Galadriel], [Imrahil, Glóin], [Imrah..."
135,32674,None,Bain,1,1,2,1,0.213750,-6628080393138316116,NaN
136,32674,None,Walda,10,3,13,78,0.440985,-8637342647242242534,"[[Thengel, Gram], [Folca, Gram], [Folca, Theng..."
137,32674,None,Thranduil,2,0,2,1,0.411049,-8637342647242242534,NaN


In [4]:
class PGStep(Step):
    def eval(self, v: Vertex):
        initLabel = 1.0
        v["prlabel"] = initLabel
        out_degree = v.out_degree()
        if out_degree > 0:
            msg = initLabel / out_degree
            v.message_outgoing_neighbours(msg)

class PGIterate(Iterate):
    def __init__(self, iterations: int, execute_messaged_only: bool, damping_factor: float = 0.85):
        super().__init__(iterations, execute_messaged_only)
        self.damping_factor = damping_factor

    def eval(self, v: Vertex):
        current_label = v["prlabel"]
        queue = v.message_queue()
        summed_queue = sum(queue)
        new_label = (1 - self.damping_factor) + self.damping_factor * summed_queue
        v["prlabel"] = new_label

        out_degree = v.out_degree()

        if out_degree > 0:
            v.message_outgoing_neighbours(new_label / out_degree)

        if abs(new_label - current_label) < 0.00001:
            v.vote_to_halt()

In [6]:
pr.shutdown()